In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"


In [ ]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
from llama_index.core.prompts import ChatPromptTemplate, ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import QueryEngineTool

# 1. Setup your index (already uploaded in LlamaCloud)
index = LlamaCloudIndex(
    name="vishal-pdf-parsing",
    project_name="Default",
    organization_id="4da7e57b-20d4-41d7-beae-de099617d5a0",
    api_key="llx-XZl46JFhSu4w33ZjvvlLQSi8vc0SlGAHzcDzdCAY5WCNsyDZ",
)

# 2. Setup the retriever and query engine (according to your UI settings)
retriever = index.as_retriever(
    search_type="default",
    search_kwargs={
        "similarity_top_k": 30,
        "node_types": ["chunk"],
        "rerank": True,
        "rerank_top_n": 6,
        "filter_mode": "accurate",
        "multimodal": False,
        "filter_condition": {
            "operator": "AND",
            "filters": []  # Add metadata filters if needed
        }
    }
)

system_prompt = """You are a virtual mentor for a Leadership Skills, taught by Professor Vishal Gupta from IIM Ahmedabad. Your role is to provide comprehensive, accurate, and relevant answers to questions about leadership skills, Adhere to the following guidelines:

1. **Exclusive Use of Course Content**: Use ONLY the information provided in the course transcripts. Do not use external knowledge or sources.
2. **Accurate Reference**: Always include the relevant week and topic title(s) in your answer, formatting it as: [Week X: Topic Title].
3. **Handling Unanswerable Questions**: If the question cannot be answered using the provided transcripts, state this clearly.
4. **Strict Non-Inference Policy**: Do not infer information not explicitly stated in the provided content.
5. **Structured and Clear Responses**: Ensure your responses are well-structured and directly quote from the transcript when appropriate.
6. **Mentor-like Tone**: Phrase your responses as a supportive virtual mentor, offering guidance and insights based on the course material.
7. **Comprehensive Answers**: Provide thorough answers, elaborating on key points and connecting ideas from different parts of the course when relevant.
8. **Consistency**: Maintain consistency in style and adherence to the guidelines throughout your responses.

Remember, accuracy and relevance to the provided course content are paramount."""

message_templates = [
    ChatMessage(role=MessageRole.SYSTEM, content=system_prompt),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query using chain-of-thought reasoning: {query_str}\n"
        ),
    ),
]
custom_prompt = ChatPromptTemplate(message_templates=message_templates)

llm = OpenAI(model="gpt-4o", temperature=0.1)

from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever,
    llm=llm,
    prompt=custom_prompt,
    chain_of_thought=True,
)

# 3. Wrap your query engine as a tool
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="query_engine_tool",
    description="Tool for querying the Leadership Skills course index.",
    return_direct=True,  # Set True to return tool output directly
)

# 4. Create an agent with the tool
agent = FunctionAgent(
    name="Vishal gupta's AI Mentor",
    description="""You are a virtual mentor for a Leadership Skills, taught by Professor Vishal Gupta from IIM Ahmedabad. Your role is to provide comprehensive, accurate, and relevant answers to questions about leadership skills, Adhere to the following guidelines:

        1. **Exclusive Use of Course Content**: Use ONLY the information provided in the course transcripts. Do not use external knowledge or sources.
        2. **Accurate Reference**: Always include the relevant week and topic title(s) in your answer, formatting it as: [Week X: Topic Title].
        3. **Handling Unanswerable Questions**: If the question cannot be answered using the provided transcripts, state this clearly.
        4. **Strict Non-Inference Policy**: Do not infer information not explicitly stated in the provided content.
        5. **Structured and Clear Responses**: Ensure your responses are well-structured and directly quote from the transcript when appropriate.
        6. **Mentor-like Tone**: Phrase your responses as a supportive virtual mentor, offering guidance and insights based on the course material.
        7. **Comprehensive Answers**: Provide thorough answers, elaborating on key points and connecting ideas from different parts of the course when relevant.
        8. **Consistency**: Maintain consistency in style and adherence to the guidelines throughout your responses.

        Remember, accuracy and relevance to the provided course content are paramount.""",
    tools=[query_engine_tool],
    llm=llm,
    system_prompt=system_prompt,
)

# 5. Run the agent with a user question
import asyncio

response = await agent.run("Hi")
print(response)


# if __name__ == "__main__":
#     asyncio.run(main())


Hello! How can I assist you with your Leadership Skills course today? If you have any questions or need guidance on specific topics, feel free to ask.


In [5]:
print(response)


Hello! How can I assist you with your Leadership Skills course today? If you have any questions or need guidance on specific topics, feel free to ask.


In [6]:
response


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Hello! How can I assist you with your Leadership Skills course today? If you have any questions or need guidance on specific topics, feel free to ask.')]), tool_calls=[], raw={'id': 'chatcmpl-BcBCCsv2Pz71vrZ3dLCeAZWyHmJae', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1748439200, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_07871e2ad8', 'usage': None}, current_agent_name='Leadership Skills Mentor')

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under a velvet sky, a gentle unicorn tiptoed through a starlit meadow, weaving dreams of magic and moonbeams for every sleeping child.
